In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

# 验证

In [ ]:
# 去云、云阴影
def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)  
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
             .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# 去云、云阴影、积雪
def maskSR2(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# 图像可视化参数
visParams = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4}

In [ ]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

In [ ]:
# # 加载样本集,PearlRiverDeltanew2020_l78,TibetPlateau_new2020_l78,daqing_new2020_l78,tianjin_new2020_l78,wuhan_new2020_l78
samples = ee.FeatureCollection('users/311605001111/wuhan_tibetan/tibetan_2016')
print("初始的样本数： ",samples.size().getInfo())

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample
filtered_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())


# RandomForest预测使用的波段，分类标签
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_samples,label,bands)

In [ ]:
# The reference image
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2016-01-01', '2016-12-31') \
       .filter(ee.Filter.eq('WRS_PATH',139)) \
       .filter(ee.Filter.eq('WRS_ROW',36))
print(l8.aggregate_array('system:id').getInfo())
Map.addLayer(l8.map(maskSR).median().clip(roi), visParams,'image mosaic')

In [ ]:
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200108', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200225', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200312', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200328', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200413', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200429', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200515', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200531', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200702', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200718', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200819', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200904', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20200920', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201006', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201022', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201123', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201209', 
'LANDSAT/LC08/C01/T1_SR/LC08_139036_20201225'

In [ ]:
# 影像的对比

# visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
visParams = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4}

image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20160402').clip(roi)
Map.addLayer(image, visParams,'image')

image_mask = maskSR(image)
Map.addLayer(image_mask, visParams,'image_snow')
image_mask2 = maskSR2(image)
Map.addLayer(image_mask2, visParams,'image_nosnow')

In [ ]:
# 分类结果的对比
classify_image = water_index(image_mask).select(bands).classify(trainedClassifier).eq(1)
Map.addLayer(classify_image,{'palette':['white','green'],'min':0,'max':1},"classify_image")

classify_image2 = water_index(image_mask2).select(bands).classify(trainedClassifier).eq(1)
Map.addLayer(classify_image2,{'palette':['white','green'],'min':0,'max':1},"classify_image2")

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8826, 34.6579,91.3626, 35.7279]
fig = plt.figure(figsize=(12, 8))
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20201225').clip(roi)
visParams = {'bands': ['B4', 'B3', 'B2'],'min': 0,'max': 3000,'gamma': 1.4}
# use cartoee to get a map
ax = cartoee.get_map(image, region=region,vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'DATE : 20201225', fontsize=28)